# Notebook 3.2 - Data Cleaning

Note: In the section Dataset choice, the respective raw dataset can be chosen (Madrid, Barcelona, or Valencia). The cleaning process is then performed on the respective city. This way, consistency in the cleaning process is maintained, while at the same time keeping data (e.g., mean values to replace outliers) specific to each city.

All transformations of variables (standardization, Box-Cox, logarithm) are implemented in the modelling step to test effect on model performance.

# Import libraries

In [1]:
import pandas as pd
import numpy as np
import re
from scipy.spatial import cKDTree
pd.options.mode.chained_assignment = None

# Choose the city

In [2]:
#Choose city ("Madrid", "Barcelona", or "Valencia")
city = "Valencia"

# Load raw dataset after merge with polyon

In [3]:
#Read corresponding file
if city == "Madrid":
    data = pd.read_csv('../../data/2_raw_idealista_data_incl_polygon/madrid_sale_with_polygon.csv')
elif city == "Barcelona":
    data = pd.read_csv('../../data/2_raw_idealista_data_incl_polygon/barcelona_sale_with_polygon.csv')
elif city == "Valencia":
    data = pd.read_csv('../../data/2_raw_idealista_data_incl_polygon/valencia_sale_with_polygon.csv')
else:
    raise ValueError("City not recognized. Please choose either 'Madrid', 'Barcelona', or 'Valencia'.")

In [4]:
data.head(20)

,ASSETID,PERIOD,PRICE,UNITPRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,HASTERRACE,HASLIFT,HASAIRCONDITIONING,...,BUILTTYPEID_1,BUILTTYPEID_2,BUILTTYPEID_3,DISTANCE_TO_CITY_CENTER,DISTANCE_TO_METRO,DISTANCE_TO_BLASCO,LONGITUDE,LATITUDE,geometry,neighborhood
0,A9651535568269959084,201812,111000.0,1480.000000,75,2,1,1,1,1,...,0,0,1,3.759093,0.756517,3.373132,-0.393538,39.500908,POINT (-0.393538331726532 39.5009075674168),Ciutat Fallera
1,A13801394133398327300,201812,169000.0,1320.312500,128,4,2,0,1,1,...,0,0,1,2.073773,0.370680,0.540594,-0.359125,39.482599,POINT (-0.359124975348099 39.4825994457946),Benimaclet
2,A8013965313053471366,201812,162000.0,1883.720930,86,2,1,0,1,1,...,0,0,1,2.099641,0.038560,0.719203,-0.362470,39.485144,POINT (-0.362469934782961 39.4851438187497),Benimaclet
3,A15308599866706500727,201812,199000.0,1792.792793,111,4,2,0,1,0,...,0,0,1,2.120917,0.524827,0.915379,-0.367417,39.487445,POINT (-0.367417112798973 39.4874451366112),Benimaclet
4,A7882254711596398998,201812,73000.0,1303.571429,56,3,1,1,1,0,...,0,0,1,3.087950,1.129985,2.009535,-0.370182,39.497057,POINT (-0.370182227980988 39.4970570379316),Els Orriols
5,A2296637762363569749,201812,45000.0,692.307692,65,3,1,0,0,0,...,0,0,1,2.723051,0.950909,1.624347,-0.369603,39.493621,POINT (-0.369602531049724 39.4936205880565),Els Orriols
6,A13832857238037339051,201812,81000.0,1012.500000,80,3,1,0,1,0,...,0,0,1,2.967437,0.936006,1.837951,-0.368560,39.495687,POINT (-0.368559503741238 39.4956870025172),Els Orriols
7,A10212260978174326059,201812,85000.0,923.913043,92,4,1,0,0,0,...,0,0,1,2.744555,1.080070,1.697090,-0.371029,39.494040,POINT (-0.371028501520023 39.4940402375392),Els Orriols
8,A12715105291622241260,201812,71000.0,612.068966,116,3,1,0,1,0,...,0,0,1,2.272737,0.942046,1.248427,-0.371248,39.489757,POINT (-0.37124805792905 39.4897570335793),Sant Antoni
9,A13467229617683088257,201812,65000.0,955.882353,68,3,1,0,1,0,...,0,0,1,2.629768,0.971491,1.544579,-0.369981,39.492827,POINT (-0.36998125894077 39.4928273815343),Els Orriols


__Create copy of dataset on which to perform manipulations__

In [5]:
data_cleaned = data.copy()

# Data cleaning

__Column names and data types__

In [6]:
# Convert all column names to uppercase for consistency
data_cleaned.columns = data_cleaned.columns.str.upper()

In [7]:
#Create consistent name for column "Distance_to_main_street" to be able to merge datasets later
if city == "Madrid":
    data_cleaned.rename(columns={"DISTANCE_TO_CASTELLANA": "DISTANCE_TO_MAIN_STREET"}, inplace=True)
elif city == "Barcelona":
    data_cleaned.rename(columns={"DISTANCE_TO_DIAGONAL": "DISTANCE_TO_MAIN_STREET"}, inplace=True)
elif city == "Valencia":
    data_cleaned.rename(columns={"DISTANCE_TO_BLASCO":  "DISTANCE_TO_MAIN_STREET"}, inplace=True)

In [8]:
# Change PERIOD to string
data_cleaned['PERIOD'] = data_cleaned['PERIOD'].astype(str)
       
#Convert columns to integers where relevant
columns_to_int = ['PRICE', 'CONSTRUCTIONYEAR', 'FLATLOCATIONID', 'ROOMNUMBER', 'CADASTRALQUALITYID', 'FLOORCLEAN']
for column in columns_to_int:
    data_cleaned[column] = pd.to_numeric(data_cleaned[column], errors='coerce').astype('Int64')
    
#Change column FLATLOCATIONID to boolean (currently external view=1 vs. internal view=2)
def assign_value(x):
    if x == 1:
        return 1
    elif x == 2:
        return 0
    else:
        return np.nan
data_cleaned['HASEXTERNALVIEW'] = data_cleaned['FLATLOCATIONID'].apply(assign_value)
data_cleaned.drop(columns=['FLATLOCATIONID'], inplace=True)

# Print the data types to verify the changes
print("Madrid Data Types:\n", data_cleaned.dtypes)

Madrid Data Types:
 ASSETID                           object
PERIOD                            object
PRICE                              Int64
UNITPRICE                        float64
CONSTRUCTEDAREA                    int64
ROOMNUMBER                         Int64
BATHNUMBER                         int64
HASTERRACE                         int64
HASLIFT                            int64
HASAIRCONDITIONING                 int64
AMENITYID                          int64
HASPARKINGSPACE                    int64
ISPARKINGSPACEINCLUDEDINPRICE      int64
PARKINGSPACEPRICE                  int64
HASNORTHORIENTATION                int64
HASSOUTHORIENTATION                int64
HASEASTORIENTATION                 int64
HASWESTORIENTATION                 int64
HASBOXROOM                         int64
HASWARDROBE                        int64
HASSWIMMINGPOOL                    int64
HASDOORMAN                         int64
HASGARDEN                          int64
ISDUPLEX                           in

__Missing values__

`NEIGHBORHOOD`: Drop rows where NEIGHBORHOOD is missing (<0.3%)

In [9]:
data_cleaned = data_cleaned.dropna(subset=['NEIGHBORHOOD'])

`CONSTRUCTIONYEAR`: Drop column, use cadastre information instead due to low share of missing values and only realistic values (see EDA)

In [10]:
data_cleaned = data_cleaned.drop(columns="CONSTRUCTIONYEAR")

`FLOORCLEAN`, `CADASTRALQUALITYID`, `HASEXTERNALVIEW` (previous `FLATLOCATIONID`): Replace missing value with median of column for respective neighborhood that the house is in

In [11]:
# Calculate median for each Neighborhood for FLOORCLEAN and CADASTRALQUALITYID and round to int
median_floorclean = data_cleaned.groupby('NEIGHBORHOOD')['FLOORCLEAN'].median().round().astype(int).reset_index().rename(columns={'FLOORCLEAN': 'MEDIAN_FLOORCLEAN'})
median_cadastral = data_cleaned.groupby('NEIGHBORHOOD')['CADASTRALQUALITYID'].median().round().astype(int).reset_index().rename(columns={'CADASTRALQUALITYID': 'MEDIAN_CADASTRALQUALITYID'})

# Merge the median values back into the original DataFrame
data_cleaned = data_cleaned.merge(median_floorclean, on='NEIGHBORHOOD', how='left')
data_cleaned = data_cleaned.merge(median_cadastral, on='NEIGHBORHOOD', how='left')

# Replace missing values with the median of the respective Neighborhood group for FLOORCLEAN and CADASTRALQUALITYID
data_cleaned['FLOORCLEAN'] = data_cleaned['FLOORCLEAN'].fillna(data_cleaned['MEDIAN_FLOORCLEAN'])
data_cleaned['CADASTRALQUALITYID'] = data_cleaned['CADASTRALQUALITYID'].fillna(data_cleaned['MEDIAN_CADASTRALQUALITYID'])

# Drop the median columns as they are no longer needed
data_cleaned = data_cleaned.drop(columns=['MEDIAN_FLOORCLEAN', 'MEDIAN_CADASTRALQUALITYID'])

In [12]:
# Calculate mode for HASEXTERNALVIEW for each NEIGHBORHOOD 
mode_values = data_cleaned.groupby('NEIGHBORHOOD')['HASEXTERNALVIEW'].apply(lambda x: x.mode()[0] if not x.mode().empty else np.nan)

# Replace missing values in HASEXTERNALVIEW with the mode of the respective NEIGHBORHOOD
data_cleaned['HASEXTERNALVIEW'] = data_cleaned.apply(
    lambda row: mode_values[row['NEIGHBORHOOD']] if pd.isna(row['HASEXTERNALVIEW']) else row['HASEXTERNALVIEW'], axis=1
)

Check that there are no missing values left

In [13]:
#Count missing values
missing_values_cleaned = data_cleaned.isnull().sum()
missing_percent_cleaned = (missing_values_cleaned / len(data_cleaned)) * 100

# Create a DataFrame to display the count and percentage of missing values
missing_data_cleaned = pd.DataFrame({
    'Missing Values': missing_values_cleaned,
    'Percentage': missing_percent_cleaned
})

# Filter to show only rows with missing values
missing_data_cleaned = missing_data_cleaned[missing_data_cleaned['Missing Values'] > 0]

if missing_data_cleaned.empty:
    print("There are no missing values left")
else:
    print("\nMissing values in the cleaned dataset:")
    print(missing_data_cleaned)

There are no missing values left


__Cleaning of outliers__

`CADDWELLINGCOUNT`: Due to some extremely high numbers likely to be an error (see EDA), we replace outliers in the 99th percentile with the column's median for the respective neighborhood

In [14]:
# Function to replace outliers with the median value for the NEIGHBORHOOD (to be reused)
def replace_outliers_with_neighborhood_median(df, column_name, threshold):
    # Find the median for each neighborhood
    neighborhood_medians = df.groupby('NEIGHBORHOOD')[column_name].transform('median')
    
    # Replace outliers with the median value
    df[column_name] = np.where(df[column_name] > threshold, neighborhood_medians, df[column_name])
    
    return df

# Identify the 99th percentile for CADDWELLINGCOUNT
percentile_99_caddwellingcount = data_cleaned['CADDWELLINGCOUNT'].quantile(0.99)

# Apply the function to replace outliers in CADDWELLINGCOUNT
data_cleaned = replace_outliers_with_neighborhood_median(data_cleaned, 'CADDWELLINGCOUNT', percentile_99_caddwellingcount)

`ROOMNUMBER` and `BATHNUMBER`: Create feature SQUAREMETERSPERROOM to identify outliers, assign average number of rooms of houses with similar size

In [15]:
# ROOMNUMBER
# Replace values equal to zero with 1 - 0 likely to be an error, and avoids division by zero
data_cleaned['ROOMNUMBER'] = data_cleaned['ROOMNUMBER'].replace(0, 1)

# Classify each house as small, medium, or large based on the ‘CONSTRUCTEDAREA’
small_threshold = data_cleaned['CONSTRUCTEDAREA'].quantile(0.33)
medium_threshold = data_cleaned['CONSTRUCTEDAREA'].quantile(0.66)

def classify_house(constructed_area):
    if (constructed_area <= small_threshold):
        return 'small'
    elif (constructed_area <= medium_threshold):
        return 'medium'
    else:
        return 'large'

data_cleaned['HOUSE_SIZE'] = data_cleaned['CONSTRUCTEDAREA'].apply(classify_house)

# Calculate average ‘ROOMNUMBER’ for each cluster
average_roomnumber = data_cleaned.groupby('HOUSE_SIZE')['ROOMNUMBER'].mean().to_dict()

# Identify outliers with new feature ‘SQUAREMETERS_PER_ROOM’
data_cleaned['SQUAREMETERS_PER_ROOM'] = data_cleaned['CONSTRUCTEDAREA'] / data_cleaned['ROOMNUMBER']
lower_threshold = data_cleaned['SQUAREMETERS_PER_ROOM'].quantile(0.01)
upper_threshold = data_cleaned['SQUAREMETERS_PER_ROOM'].quantile(0.99)
outliers = (data_cleaned['SQUAREMETERS_PER_ROOM'] < lower_threshold) | (data_cleaned['SQUAREMETERS_PER_ROOM'] > upper_threshold)

# Calculate the number of outliers
num_outliers = outliers.sum()

# Replace ‘ROOMNUMBER’ for identified outliers with average ‘ROOMNUMBER’ of the respective size cluster
data_cleaned.loc[outliers, 'ROOMNUMBER'] = data_cleaned.loc[outliers, 'HOUSE_SIZE'].map(average_roomnumber).astype(int)

# Print the number of outliers replaced
print(f"Number of outliers replaced: {num_outliers}")

# Print the distribution of ROOMNUMBER after replacement
print("ROOMNUMBER distribution after replacement:")
print(data_cleaned['ROOMNUMBER'].describe())


Number of outliers replaced: 633
ROOMNUMBER distribution after replacement:
count     33578.0
mean     3.068587
std      0.958952
min           1.0
25%           3.0
50%           3.0
75%           4.0
max          15.0
Name: ROOMNUMBER, dtype: Float64


In [16]:
# BATHNUMBER
# Replace values equal to zero in BATHNUMBER with 1 - 0 likely to be an error, and avoids division by zero
data_cleaned['BATHNUMBER'] = data_cleaned['BATHNUMBER'].replace(0, 1)

# Calculate average ‘BATHNUMBER’ for each cluster
average_bathnumber = data_cleaned.groupby('HOUSE_SIZE')['BATHNUMBER'].mean().to_dict()

# Identify outliers with new feature ‘SQUAREMETERS_PER_BATH’
data_cleaned['SQUAREMETERS_PER_BATH'] = data_cleaned['CONSTRUCTEDAREA'] / data_cleaned['BATHNUMBER']
lower_threshold = data_cleaned['SQUAREMETERS_PER_BATH'].quantile(0.01)
upper_threshold = data_cleaned['SQUAREMETERS_PER_BATH'].quantile(0.99)
outliers = (data_cleaned['SQUAREMETERS_PER_BATH'] < lower_threshold) | (data_cleaned['SQUAREMETERS_PER_BATH'] > upper_threshold)


# Replace ‘BATHNUMBER’ for identified outliers with average ‘BATHNUMBER’ of the respective size cluster
data_cleaned.loc[outliers, 'BATHNUMBER'] = data_cleaned.loc[outliers, 'HOUSE_SIZE'].map(average_bathnumber).astype(int)


# Print the number of outliers replaced
print(f"Number of outliers replaced: {num_outliers}")

# Print the distribution of BATHNUMBER after replacement
print("BATHNUMBER distribution after replacement:")
print(data_cleaned['BATHNUMBER'].describe())

Number of outliers replaced: 633
BATHNUMBER distribution after replacement:
count    33578.000000
mean         1.582911
std          0.627892
min          1.000000
25%          1.000000
50%          2.000000
75%          2.000000
max         12.000000
Name: BATHNUMBER, dtype: float64


`PARKINGSPACEPRICE`: Create a new column HASPARKINGSPACE_CLEANED that takes 1 if the previous column HASPARKINGSPACE is equal to 1, or if the price for parking is above 1. For the price itself, replace extremely high values likely to be an error (in the 99.9th percentile as very few outliers only) with the average parking space price in the respective neighborhood

In [17]:
# Identify the 99.9th percentile for PARKINGSPACEPRICE
percentile_999_parkingspaceprice = data_cleaned['PARKINGSPACEPRICE'].quantile(0.999)

# Apply the function to replace outliers in PARKINGSPACEPRICE
data_cleaned = replace_outliers_with_neighborhood_median(data_cleaned, 'PARKINGSPACEPRICE', percentile_999_parkingspaceprice)

In [18]:
data_cleaned["PARKINGSPACEPRICE"].describe()

count    33578.000000
mean       659.506016
std       4154.969207
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max      50001.000000
Name: PARKINGSPACEPRICE, dtype: float64

Drop auxiliary columns crated for outlier identification

In [19]:
data_cleaned = data_cleaned.drop(columns=['HOUSE_SIZE', 'SQUAREMETERS_PER_BATH', 'SQUAREMETERS_PER_ROOM'])

__Aggregate duplicates in assets__

In [20]:
data_cleaned.head()

,ASSETID,PERIOD,PRICE,UNITPRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,HASTERRACE,HASLIFT,HASAIRCONDITIONING,...,BUILTTYPEID_2,BUILTTYPEID_3,DISTANCE_TO_CITY_CENTER,DISTANCE_TO_METRO,DISTANCE_TO_MAIN_STREET,LONGITUDE,LATITUDE,GEOMETRY,NEIGHBORHOOD,HASEXTERNALVIEW
0,A9651535568269959084,201812,111000,1480.000000,75,2,1,1,1,1,...,0,1,3.759093,0.756517,3.373132,-0.393538,39.500908,POINT (-0.393538331726532 39.5009075674168),Ciutat Fallera,1.0
1,A13801394133398327300,201812,169000,1320.312500,128,4,2,0,1,1,...,0,1,2.073773,0.370680,0.540594,-0.359125,39.482599,POINT (-0.359124975348099 39.4825994457946),Benimaclet,1.0
2,A8013965313053471366,201812,162000,1883.720930,86,2,1,0,1,1,...,0,1,2.099641,0.038560,0.719203,-0.362470,39.485144,POINT (-0.362469934782961 39.4851438187497),Benimaclet,1.0
3,A15308599866706500727,201812,199000,1792.792793,111,4,2,0,1,0,...,0,1,2.120917,0.524827,0.915379,-0.367417,39.487445,POINT (-0.367417112798973 39.4874451366112),Benimaclet,1.0
4,A7882254711596398998,201812,73000,1303.571429,56,2,1,1,1,0,...,0,1,3.087950,1.129985,2.009535,-0.370182,39.497057,POINT (-0.370182227980988 39.4970570379316),Els Orriols,1.0


As aligned with Idealista, the entries in the data are not necessarily sequential. Therefore, we take the average / mode for each ASSETID, depending on the column's type.

In [21]:
data_cleaned.columns

Index(['ASSETID', 'PERIOD', 'PRICE', 'UNITPRICE', 'CONSTRUCTEDAREA',
       'ROOMNUMBER', 'BATHNUMBER', 'HASTERRACE', 'HASLIFT',
       'HASAIRCONDITIONING', 'AMENITYID', 'HASPARKINGSPACE',
       'ISPARKINGSPACEINCLUDEDINPRICE', 'PARKINGSPACEPRICE',
       'HASNORTHORIENTATION', 'HASSOUTHORIENTATION', 'HASEASTORIENTATION',
       'HASWESTORIENTATION', 'HASBOXROOM', 'HASWARDROBE', 'HASSWIMMINGPOOL',
       'HASDOORMAN', 'HASGARDEN', 'ISDUPLEX', 'ISSTUDIO', 'ISINTOPFLOOR',
       'FLOORCLEAN', 'CADCONSTRUCTIONYEAR', 'CADMAXBUILDINGFLOOR',
       'CADDWELLINGCOUNT', 'CADASTRALQUALITYID', 'BUILTTYPEID_1',
       'BUILTTYPEID_2', 'BUILTTYPEID_3', 'DISTANCE_TO_CITY_CENTER',
       'DISTANCE_TO_METRO', 'DISTANCE_TO_MAIN_STREET', 'LONGITUDE', 'LATITUDE',
       'GEOMETRY', 'NEIGHBORHOOD', 'HASEXTERNALVIEW'],
      dtype='object')

In [22]:
# Define for which columns we take the mean
mean_columns = ['PRICE', 'UNITPRICE', 'CONSTRUCTEDAREA', 'ROOMNUMBER', 'BATHNUMBER', 'PARKINGSPACEPRICE', 'FLOORCLEAN', 'CADMAXBUILDINGFLOOR', 'CADCONSTRUCTIONYEAR',
                'CADDWELLINGCOUNT', 'DISTANCE_TO_CITY_CENTER', 'DISTANCE_TO_METRO', 'DISTANCE_TO_MAIN_STREET', 'LONGITUDE', 'LATITUDE']

# Define for which columns we take the mode 
mode_columns = ['AMENITYID', 'HASPARKINGSPACE', 'ISPARKINGSPACEINCLUDEDINPRICE', 'HASTERRACE', 'HASLIFT', 'HASAIRCONDITIONING', 'HASNORTHORIENTATION', 'HASSOUTHORIENTATION', 
                'HASEASTORIENTATION', 'HASWESTORIENTATION', 'HASBOXROOM', 'HASWARDROBE', 'HASSWIMMINGPOOL', 'HASDOORMAN', 'HASGARDEN', 'ISDUPLEX', 
                'ISSTUDIO', 'HASEXTERNALVIEW', 'ISINTOPFLOOR', 'CADASTRALQUALITYID', 'BUILTTYPEID_1', 'BUILTTYPEID_2', 'BUILTTYPEID_3', 'NEIGHBORHOOD', 'GEOMETRY']

# Group by 'ASSETID' and 'PERIOD' and calculate the mean for mean_columns
mean_df = data_cleaned.groupby(['ASSETID', 'PERIOD'])[mean_columns].mean().reset_index()

# Group by 'ASSETID' and 'PERIOD' and calculate the mode for mode_columns
def calculate_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

mode_df = data_cleaned.groupby(['ASSETID', 'PERIOD'])[mode_columns].agg(lambda x: calculate_mode(x)).reset_index()

# Merge the two DataFrames on 'ASSETID' and 'PERIOD'
result = pd.merge(mean_df, mode_df, on=['ASSETID', 'PERIOD'])

# Drop any duplicate columns that might arise from the merge
result = result.loc[:, ~result.columns.duplicated()]

# Assign the final result to data_cleaned
data_cleaned = result

# Display the first few rows of the cleaned data
data_cleaned.head()

,ASSETID,PERIOD,PRICE,UNITPRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,PARKINGSPACEPRICE,FLOORCLEAN,CADMAXBUILDINGFLOOR,...,ISDUPLEX,ISSTUDIO,HASEXTERNALVIEW,ISINTOPFLOOR,CADASTRALQUALITYID,BUILTTYPEID_1,BUILTTYPEID_2,BUILTTYPEID_3,NEIGHBORHOOD,GEOMETRY
0,A10000434603646497633,201803,126500.0,1807.142857,70.0,2.0,1.0,1.0,1.0,3.0,...,0,0,1.0,0,3,0,0,1,El Cabanyal-El Canyamelar,POINT (-0.330160605703139 39.4635957886042)
1,A10001334147587469388,201812,630000.0,3165.829146,199.0,4.0,2.0,1.0,10.0,14.0,...,0,0,1.0,1,3,0,0,1,Ciutat de les Arts i de les Ciencies,POINT (-0.357513093932358 39.4566986180307)
2,A10002599312155392987,201812,215000.0,2150.000000,100.0,3.0,2.0,1.0,3.0,7.0,...,0,0,1.0,0,5,0,0,1,El Grau,POINT (-0.334319928169434 39.4630246214926)
3,A10002648121225460937,201812,240000.0,2142.857143,112.0,3.0,2.0,1.0,5.666667,6.0,...,0,0,1.0,1,5,0,0,1,Nou Moles,POINT (-0.401599173510795 39.4696293401522)
4,A10002658173109908582,201812,1160000.0,4013.840830,289.0,5.0,4.0,1.0,5.0,14.0,...,0,0,1.0,0,1,0,0,1,El Pla del Remei,POINT (-0.367873821982046 39.472563980498)


In [23]:
# Check that no duplicates in ASSETID are left
num_duplicates = data_cleaned.duplicated(subset=['ASSETID']).sum()
print("Number of duplicates in the ASSETID column:", num_duplicates)

# Analyze effect of aggregation on dataset
num_rows_before_aggregation = len(data)
print("Number of rows before aggregation:", num_rows_before_aggregation)
num_rows_after_aggregation = len(data_cleaned)
print("Number of rows after aggregation:", num_rows_after_aggregation)

Number of duplicates in the ASSETID column: 0
Number of rows before aggregation: 33622
Number of rows after aggregation: 27364


__Change order of columns for interpretability and correct data types__

In [24]:
# Round columns that should not have a decimal number to nearest integer and convert to int
columns_to_round = ['ROOMNUMBER', 'BATHNUMBER', 'FLOORCLEAN', 'CADMAXBUILDINGFLOOR', 'CADCONSTRUCTIONYEAR', 'CADDWELLINGCOUNT', 'PRICE', 'CONSTRUCTEDAREA']

data_cleaned[columns_to_round] = data_cleaned[columns_to_round].round().astype(int)

# Desired order of columns
desired_order = [
    'ASSETID', 'PERIOD', 'PRICE', 'UNITPRICE', 'CONSTRUCTEDAREA',
    'ROOMNUMBER', 'BATHNUMBER',  'AMENITYID', 'HASPARKINGSPACE',
    'ISPARKINGSPACEINCLUDEDINPRICE', 'PARKINGSPACEPRICE', 'HASTERRACE', 'HASLIFT',
    'HASAIRCONDITIONING',  'HASNORTHORIENTATION', 'HASSOUTHORIENTATION', 'HASEASTORIENTATION',
    'HASWESTORIENTATION', 'HASBOXROOM', 'HASWARDROBE', 'HASSWIMMINGPOOL',
    'HASDOORMAN', 'HASGARDEN', 'ISDUPLEX', 'ISSTUDIO', 'HASEXTERNALVIEW', 'ISINTOPFLOOR',  'FLOORCLEAN', 'CADMAXBUILDINGFLOOR',  'CADCONSTRUCTIONYEAR',  'CADDWELLINGCOUNT','CADASTRALQUALITYID', 'BUILTTYPEID_1',  'BUILTTYPEID_2', 'BUILTTYPEID_3','DISTANCE_TO_CITY_CENTER', 'DISTANCE_TO_METRO',  'DISTANCE_TO_MAIN_STREET', 'LONGITUDE','LATITUDE',  'GEOMETRY', 'NEIGHBORHOOD'
]

# Reorder the columns
data_cleaned = data_cleaned[desired_order]

__Inspect dataset after cleaning__

In [25]:
data_cleaned.head(10)

,ASSETID,PERIOD,PRICE,UNITPRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,ISPARKINGSPACEINCLUDEDINPRICE,...,BUILTTYPEID_1,BUILTTYPEID_2,BUILTTYPEID_3,DISTANCE_TO_CITY_CENTER,DISTANCE_TO_METRO,DISTANCE_TO_MAIN_STREET,LONGITUDE,LATITUDE,GEOMETRY,NEIGHBORHOOD
0,A10000434603646497633,201803,126500,1807.142857,70,2,1,3,0,0,...,0,0,1,4.044289,0.734973,0.854680,-0.330256,39.463535,POINT (-0.330160605703139 39.4635957886042),El Cabanyal-El Canyamelar
1,A10001334147587469388,201812,630000,3165.829146,199,4,2,3,1,1,...,0,0,1,2.193673,0.724361,2.102044,-0.357513,39.456699,POINT (-0.357513093932358 39.4566986180307),Ciutat de les Arts i de les Ciencies
2,A10002599312155392987,201812,215000,2150.000000,100,3,2,3,0,0,...,0,0,1,3.711768,0.423130,0.800718,-0.334320,39.463025,POINT (-0.334319928169434 39.4630246214926),El Grau
3,A10002648121225460937,201812,240000,2142.857143,112,3,2,1,0,0,...,0,0,1,2.175821,0.378732,3.256303,-0.401935,39.469964,POINT (-0.401599173510795 39.4696293401522),Nou Moles
4,A10002658173109908582,201812,1160000,4013.840830,289,5,4,2,0,0,...,0,0,1,0.799293,0.186091,0.770513,-0.367956,39.472377,POINT (-0.367873821982046 39.472563980498),El Pla del Remei
5,A10003205125900904965,201803,205000,2050.000000,100,3,2,3,1,1,...,0,0,1,1.180368,0.538609,2.560039,-0.387754,39.463568,POINT (-0.38723918226041 39.4632444439486),La Fontsanta
6,A1000367650272331670,201809,60000,882.352941,68,2,1,3,0,0,...,0,1,0,3.004559,0.947917,1.874068,-0.368500,39.496019,POINT (-0.368500467189448 39.4960189893973),Els Orriols
7,A10003769364183389417,201809,146000,1269.565217,115,3,2,3,0,0,...,0,0,1,1.966580,0.714995,3.294269,-0.397264,39.462145,POINT (-0.397263705516888 39.4621450807861),Safranar
8,A10004094816451748467,201806,97000,1102.272727,88,3,1,3,0,0,...,0,0,1,1.648895,0.948719,1.474654,-0.382081,39.483966,POINT (-0.382080539041303 39.4839657112742),Marxalenes
9,A10004671247064458424,201812,94000,1287.671233,73,3,1,2,0,0,...,0,0,1,4.216395,1.869873,1.289646,-0.329772,39.481037,POINT (-0.329772432204423 39.4810367788998),Playa de la Malvarrosa


# Add zip code to enable feature engineering

__Read the respective dataset of metro with addresses created with API before__

In [26]:
if city == "Madrid":
    metro_with_addresses = pd.read_csv('../../data/3_external_data/pois_incl_address/madrid_pois_address.csv')
elif city == "Barcelona":
    metro_with_addresses = pd.read_csv('../../data/3_external_data/pois_incl_address/barcelona_pois_address.csv')
elif city == "Valencia":
    metro_with_addresses = pd.read_csv('../../data/3_external_data/pois_incl_address/valencia_pois_address.csv')

__Add address of closest metro to cleaned data and extract zip code__

In [27]:
# Build KDTrees for the metro coordinates
metro_coords = metro_with_addresses[['Lat', 'Lon']].values
tree = cKDTree(metro_coords)

# Function to find the closest metro address
def find_closest_address(row, tree, metro_data):
    city_coords = (row['LATITUDE'], row['LONGITUDE'])
    distance, index = tree.query(city_coords)
    closest_row = metro_data.iloc[index]
    return closest_row['Address']

# Add address_closest_metro column to cleaned_data
data_cleaned['ADDRESS_CLOSEST_METRO'] = data_cleaned.apply(
    lambda row: find_closest_address(row, tree, metro_with_addresses), axis=1)

# Function to extract ZIP code
def extract_zip_code(address):
    match = re.search(r'\b\d{5}\b', address)
    return match.group(0) if match else None

# Add ZIP_code_metro column to cleaned data
data_cleaned['ZIP_CODE'] = data_cleaned['ADDRESS_CLOSEST_METRO'].apply(extract_zip_code)

#Change data type of zip code to string
data_cleaned['ZIP_CODE'] = data_cleaned['ZIP_CODE'].astype(str)

# Categorical encoding

Note: Target encoding of NEIGHBORHOOD / ZIP_CODE is implemented in modelling step to avoid data leakage from training to test set

In [28]:
#Ignore depreciation warning
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

#PERIOD: One-hot encoding and turn boolean column to int for consistency
data_cleaned = pd.get_dummies(data_cleaned, columns=['PERIOD'], prefix='PERIOD')
for col in data_cleaned.columns:
    if 'PERIOD_' in col:
        data_cleaned[col] = data_cleaned[col].astype(int)

#CADASTRALQUALITYID: Reverse order by subtracting the original ID from the maximum value 9 (for interpretability)
data_cleaned['CADASTRALQUALITYID'] = 9 - data_cleaned['CADASTRALQUALITYID']

# Inspect dataset after cleaning, categorical encoding, and adding zip codes

In [29]:
data_cleaned.head(10)

,ASSETID,PRICE,UNITPRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,ISPARKINGSPACEINCLUDEDINPRICE,PARKINGSPACEPRICE,...,LONGITUDE,LATITUDE,GEOMETRY,NEIGHBORHOOD,ADDRESS_CLOSEST_METRO,ZIP_CODE,PERIOD_201803,PERIOD_201806,PERIOD_201809,PERIOD_201812
0,A10000434603646497633,126500,1807.142857,70,2,1,3,0,0,1.0,...,-0.330256,39.463535,POINT (-0.330160605703139 39.4635957886042),El Cabanyal-El Canyamelar,"unnamed road, 46011 Valencia, Spain",46011,1,0,0,0
1,A10001334147587469388,630000,3165.829146,199,4,2,3,1,1,1.0,...,-0.357513,39.456699,POINT (-0.357513093932358 39.4566986180307),Ciutat de les Arts i de les Ciencies,"Amado Granell-Montolivet, Avinguda d'Amado Gra...",46005,0,0,0,1
2,A10002599312155392987,215000,2150.000000,100,3,2,3,0,0,1.0,...,-0.334320,39.463025,POINT (-0.334319928169434 39.4630246214926),El Grau,"Marítim, Carrer de Jeroni de Montsoriu, 46022 ...",46022,0,0,0,1
3,A10002648121225460937,240000,2142.857143,112,3,2,1,0,0,1.0,...,-0.401935,39.469964,POINT (-0.401599173510795 39.4696293401522),Nou Moles,"Avinguda del Cid, 46018 Valencia, Spain",46018,0,0,0,1
4,A10002658173109908582,1160000,4013.840830,289,5,4,2,0,0,1.0,...,-0.367956,39.472377,POINT (-0.367873821982046 39.472563980498),El Pla del Remei,"Alameda, Pont de l'Exposició, 46004 Valencia, ...",46004,0,0,0,1
5,A10003205125900904965,205000,2050.000000,100,3,2,3,1,1,1.0,...,-0.387754,39.463568,POINT (-0.38723918226041 39.4632444439486),La Fontsanta,"unnamed road, 46007 Valencia, Spain",46007,1,0,0,0
6,A1000367650272331670,60000,882.352941,68,2,1,3,0,0,1.0,...,-0.368500,39.496019,POINT (-0.368500467189448 39.4960189893973),Els Orriols,"Machado, Carrer d'Emili Baró, 46020 Valencia, ...",46020,0,0,1,0
7,A10003769364183389417,146000,1269.565217,115,3,2,3,0,0,1.0,...,-0.397264,39.462145,POINT (-0.397263705516888 39.4621450807861),Safranar,"Avinguda del Cid, 46018 Valencia, Spain",46018,0,0,1,0
8,A10004094816451748467,97000,1102.272727,88,3,1,3,0,0,1.0,...,-0.382081,39.483966,POINT (-0.382080539041303 39.4839657112742),Marxalenes,"Túria, Pont de les Glòries Valencianes, 46008 ...",46008,0,1,0,0
9,A10004671247064458424,94000,1287.671233,73,3,1,2,0,0,1.0,...,-0.329772,39.481037,POINT (-0.329772432204423 39.4810367788998),Playa de la Malvarrosa,"unnamed road, 46011 Valencia, Spain",46011,0,0,0,1


# Drop features not relevant for modelling

- Drop UNITPRICE as the variable is a result of the target variable PRICE
- Drop ASSETID as it is a unique identifier only
- Drop GEOMETRY as it is a repetition of LONGITUDE and LATITUDE
- Drop ISPARKINGSPACEINCLUDEDINPRICE as EDA showed that HASPARKINGPRICE and ISPARKINGSPACEINCLUDEDINPRICE are equal to each other for all assets for all 3 cities.
- Keep for now for feature engineering, to be dropped for modelling: `ASSETID`, `NEIGHBORHOOD`, and `ZIP_CODE`

In [30]:
data_cleaned = data_cleaned.drop(columns=['UNITPRICE', 'GEOMETRY', 'ISPARKINGSPACEINCLUDEDINPRICE', 'ADDRESS_CLOSEST_METRO'])

In [31]:
data_cleaned.head()

,ASSETID,PRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,AMENITYID,HASPARKINGSPACE,PARKINGSPACEPRICE,HASTERRACE,HASLIFT,...,DISTANCE_TO_METRO,DISTANCE_TO_MAIN_STREET,LONGITUDE,LATITUDE,NEIGHBORHOOD,ZIP_CODE,PERIOD_201803,PERIOD_201806,PERIOD_201809,PERIOD_201812
0,A10000434603646497633,126500,70,2,1,3,0,1.0,0,0,...,0.734973,0.854680,-0.330256,39.463535,El Cabanyal-El Canyamelar,46011,1,0,0,0
1,A10001334147587469388,630000,199,4,2,3,1,1.0,1,1,...,0.724361,2.102044,-0.357513,39.456699,Ciutat de les Arts i de les Ciencies,46005,0,0,0,1
2,A10002599312155392987,215000,100,3,2,3,0,1.0,1,1,...,0.423130,0.800718,-0.334320,39.463025,El Grau,46022,0,0,0,1
3,A10002648121225460937,240000,112,3,2,1,0,1.0,0,1,...,0.378732,3.256303,-0.401935,39.469964,Nou Moles,46018,0,0,0,1
4,A10002658173109908582,1160000,289,5,4,2,0,1.0,0,0,...,0.186091,0.770513,-0.367956,39.472377,El Pla del Remei,46004,0,0,0,1


# Write cleaned dataset to csv as input for notebooks 4 - Feature engineering

In [32]:
if city == "Madrid":
    data_cleaned.to_csv("../../data/4_data_cleaned/madrid_cleaned_base_features.csv", index=False)
elif city == "Barcelona":
    data_cleaned.to_csv("../../data/4_data_cleaned/barcelona_cleaned_base_features.csv", index=False)
elif city == "Valencia":
    data_cleaned.to_csv("../../data/4_data_cleaned/valencia_cleaned_base_features.csv", index=False)